In [1]:
using PorousMaterials
using JLD2
using PyPlot
using PyCall
using CSV
using DataFrames

PyPlot.matplotlib.style.use("ggplot")
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["font.size"] = 16;

In [2]:
# read in lines to get COF filenames
filename = joinpath(pwd(), "./descriptors/cof_names.txt")
cof_name_file = open(filename)
cof_names = readlines(cof_name_file)
close(cof_name_file)

In [3]:
# load 
chem_props_filename = joinpath(pwd(), "descriptors/chemical_properties.csv")
geo_props_filename  = joinpath(pwd(), "descriptors/geometric_properties.csv")

chemical_properties  = CSV.read(chem_props_filename, DataFrame)
geometric_properties = CSV.read(geo_props_filename, DataFrame)

,crystal_name,pore_diameter_Å,void_fraction,surface_area_m²g⁻¹
,String,Float64,Float64,Float64
1,05000N2_ddec.cif,3.84928,0.2,4175.64
2,05001N2_ddec.cif,19.7302,0.6,3156.96
3,07000N2_ddec.cif,26.1625,0.8,3407.23
4,07001N2_ddec.cif,7.82083,0.4,2565.59
5,07002N2_ddec.cif,14.8469,0.4,3345.75
6,07010N3_ddec.cif,9.33026,0.7,5261.66
7,07011N3_ddec.cif,9.94867,0.7,5452.36
8,07012N3_ddec.cif,18.946,0.9,5335.37
9,07013N3_ddec.cif,27.193,0.9,5190.06


In [4]:
? @load

```
@load filename var1 [var2 ...]
```

Load one or more variables `var1,...` from JLD2 file `filename` into the current scope and return a vector of the loaded variable names.

For interactive use, the form `@load "somefile.jld2"` will load all variables from `"somefile.jld2"` into the current scope. This form only supports literal file names and should be avoided in more permanent code so that it's clear where the variables come from.

# Example

To load the variables `hello` and `foo` from the file example.jld2, use

```
@load "example.jld2" hello foo
```


In [5]:
adsorbates = Molecule.(["Kr", "Xe"])
temperature = 298.0 # K
ljff = LJForceField("UFF")
nb_ins_per_vol = 500

# for some reason these ones aren't working.. look into it
skip_for_now = ["07012N3_ddec.cif"]

1-element Vector{String}:
 "07012N3_ddec.cif"

In [6]:
henry_calculations = Dict{Tuple{String, Symbol}, Dict{String, Any}}()

for cof_name in cof_names
    xtal = Crystal(cof_name; check_neutrality=false)
    if xtal.name in skip_for_now
        continue
    end
    
    for molecule in adsorbates

        henry_filename = henry_result_savename(xtal, molecule, temperature, ljff, nb_ins_per_vol)
        
        @load joinpath(PorousMaterials.rc[:paths][:simulations], henry_filename) results
        
        henry_calculations[(xtal.name, molecule.species)] = results
    end
end

In [7]:
names(geometric_properties)

4-element Vector{String}:
 "crystal_name"
 "pore_diameter_Å"
 "void_fraction"
 "surface_area_m²g⁻¹"

In [11]:
?findfirst()

```
findfirst(A)
```

Return the index or key of the first `true` value in `A`. Return `nothing` if no such value is found. To search for other kinds of values, pass a predicate as the first argument.

Indices or keys are of the same type as those returned by [`keys(A)`](@ref) and [`pairs(A)`](@ref).

# Examples

```jldoctest
julia> A = [false, false, true, false]
4-element Vector{Bool}:
 0
 0
 1
 0

julia> findfirst(A)
3

julia> findfirst(falses(3)) # returns nothing, but not printed in the REPL

julia> A = [false false; true false]
2×2 Matrix{Bool}:
 0  0
 1  0

julia> findfirst(A)
CartesianIndex(2, 1)
```

---

```
findfirst(predicate::Function, A)
```

Return the index or key of the first element of `A` for which `predicate` returns `true`. Return `nothing` if there is no such element.

Indices or keys are of the same type as those returned by [`keys(A)`](@ref) and [`pairs(A)`](@ref).

# Examples

```jldoctest
julia> A = [1, 4, 2, 2]
4-element Vector{Int64}:
 1
 4
 2
 2

julia> findfirst(iseven, A)
2

julia> findfirst(x -> x>10, A) # returns nothing, but not printed in the REPL

julia> findfirst(isequal(4), A)
2

julia> A = [1 4; 2 2]
2×2 Matrix{Int64}:
 1  4
 2  2

julia> findfirst(iseven, A)
CartesianIndex(2, 1)
```

---

```
findfirst(pattern::AbstractString, string::AbstractString)
findfirst(pattern::AbstractPattern, string::String)
```

Find the first occurrence of `pattern` in `string`. Equivalent to [`findnext(pattern, string, firstindex(s))`](@ref).

# Examples

```jldoctest
julia> findfirst("z", "Hello to the world") # returns nothing, but not printed in the REPL

julia> findfirst("Julia", "JuliaLang")
1:5
```

---

```
findfirst(ch::AbstractChar, string::AbstractString)
```

Find the first occurrence of character `ch` in `string`.

!!! compat "Julia 1.3"
    This method requires at least Julia 1.3.


# Examples

```jldoctest
julia> findfirst('a', "happy")
2

julia> findfirst('z', "happy") === nothing
true
```

---

```
findfirst(pattern::AbstractVector{<:Union{Int8,UInt8}},
          A::AbstractVector{<:Union{Int8,UInt8}})
```

Find the first occurrence of sequence `pattern` in vector `A`.

!!! compat "Julia 1.6"
    This method requires at least Julia 1.6.


# Examples

```jldoctest
julia> findfirst([0x52, 0x62], [0x40, 0x52, 0x62, 0x63])
2:3
```


## Plotting Functions

In [8]:
function plot_vfxn_vs_henry(results_dict::Dict{Tuple{String, Symbol}, Dict{String, Any}}, 
                            geo_props::DataFrame, gas::Molecule)
    # do things
    figure()
    plt_vals = Dict{String, Array{Float64, 1}}()
    for key in keys(results_dict)
        xtal_name = key[1]
        
        if key[2] != gas.species
            continue
        end
        h = results_dict[key]["henry coefficient [mol/(m³-bar)]"]
        v = geo_props[findfirst(geo_props."crystal_name" .== xtal_name),
                    "void_fraction"] # df[df.A .> 500, :]
        plt_vals[key[1]] = [h, v]
    end
    
    x = [plt_vals[key][1] for key in keys(plt_vals)]
    y = [plt_vals[key][2] for key in keys(plt_vals)]
    scatter(x, y)
    xlabel("Henry coefficient")
    ylabel("void fraction")
    tight_layout()
end

plot_vfxn_vs_henry (generic function with 1 method)

In [9]:
plot_vfxn_vs_henry(henry_calculations, geometric_properties, adsorbates[2])

LoadError: MethodError: [0mCannot `convert` an object of type [92mVector{Float64}[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::ColorTypes.Gray24[39m) where T<:Real at /home/ng/.julia/packages/ColorTypes/7OlxI/src/conversions.jl:114
[0m  convert(::Type{T}, [91m::ColorTypes.Gray[39m) where T<:Real at /home/ng/.julia/packages/ColorTypes/7OlxI/src/conversions.jl:113
[0m  convert(::Type{S}, [91m::CategoricalArrays.CategoricalValue[39m) where S<:Union{AbstractChar, AbstractString, Number} at /home/ng/.julia/packages/CategoricalArrays/Fr04b/src/value.jl:79
[0m  ...